In [13]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os

# Chall 2
labels = {
    0: 'bird',
    1: 'eight',
    2: 'falcon',
    3: 'five',
    4: 'four',
    5: 'nine',
    6: 'one',
    7: 'seven',
    8: 'six',
    9: 'snake',
    10: 'three',
    11: 'two',
    12: 'zero'}

# Chall 4
labels = {
    0: 'bird',
    1: 'cat',
    2: 'chicken',
    3: 'dog',
    4: 'down',
    5: 'eight',
    6: 'falcon',
    7: 'five',
    8: 'four',
    9: 'go',
    10: 'left',
    11: 'nine',
    12: 'one',
    13: 'right',
    14: 'seven',
    15: 'six',
    16: 'snake',
    17: 'stop',
    18: 'three',
    19: 'two',
    20: 'up',
    21: 'zero'}

# Chall 6
labels = {
    0: 'backward',
    1: 'bird',
    2: 'cat',
    3: 'chicken',
    4: 'crocodile',
    5: 'dinosaur',
    6: 'dog',
    7: 'down',
    8: 'eight',
    9: 'elephant',
    10: 'falcon',
    11: 'five',
    12: 'forward',
    13: 'four',
    14: 'go',
    15: 'left',
    16: 'nine',
    17: 'one',
    18: 'right',
    19: 'seven',
    20: 'six',
    21: 'snake',
    22: 'stop',
    23: 'three',
    24: 'two',
    25: 'up',
    26: 'zero'}


# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=r"models_data_v2_12_labels\my_model\tflite_non_stream/non_stream.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# audio, _ = tf.audio.decode_wav(tf.io.read_file('s1_test_release/s1_test_0003.wav'))  # snake
# waveform = tf.expand_dims(tf.squeeze(audio, axis=-1), axis=-2)
# # Test the model on random input data.
# input_shape = input_details[0]['shape']
# input_data = waveform
# interpreter.set_tensor(input_details[0]['index'], input_data)
#
# interpreter.invoke()
#
# # The function `get_tensor()` returns a copy of the tensor data.
# # Use `tensor()` in order to get a pointer to the tensor.
# output_data = interpreter.get_tensor(output_details[0]['index'])
# softmax_output = tf.nn.softmax(output_data)[0].numpy()
# print(softmax_output)
# assert len(softmax_output) == len(labels)
# print(labels[np.argmax(softmax_output)])

predicted_labels, softmax_outputs, filenames = [], [], []
test_files = tf.io.gfile.glob('challenge_6_test_dataset_normalized/*')
print('Test set size', len(test_files))
i = 0
print('Predicting...')
for audio in test_files:
    audio, _ = tf.audio.decode_wav(tf.io.read_file(audio))
    waveform = tf.expand_dims(tf.squeeze(audio, axis=-1), axis=-2)
    filenames.append(os.path.basename(test_files[i].replace('_normalized', '')))
    interpreter.set_tensor(input_details[0]['index'], waveform)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    softmax_output = tf.nn.softmax(output_data)[0].numpy()
    softmax_outputs.append(softmax_output)
    predicted_labels.append(labels[np.argmax(softmax_output)])
    i+=1

df = pd.DataFrame(list(zip(filenames, predicted_labels)))
df.to_csv('challenge_6_team_Tensor is not flowing.csv', index=False, header=False)  # tested submission file format passed
print(df)

df = pd.DataFrame(list(zip(filenames, softmax_outputs)))  # for ensemble
df.to_csv('softmax_outputs.csv', index=False, header=False)
print(df)

Test set size 2657
Predicting...
                              0          1
0     challenge_6_test_0000.wav       bird
1     challenge_6_test_0001.wav      three
2     challenge_6_test_0002.wav  crocodile
3     challenge_6_test_0003.wav        dog
4     challenge_6_test_0004.wav    chicken
...                         ...        ...
2652  challenge_6_test_2695.wav        cat
2653  challenge_6_test_2696.wav      three
2654  challenge_6_test_2697.wav      eight
2655  challenge_6_test_2698.wav  crocodile
2656  challenge_6_test_2699.wav  crocodile

[2657 rows x 2 columns]
                              0  \
0     challenge_6_test_0000.wav   
1     challenge_6_test_0001.wav   
2     challenge_6_test_0002.wav   
3     challenge_6_test_0003.wav   
4     challenge_6_test_0004.wav   
...                         ...   
2652  challenge_6_test_2695.wav   
2653  challenge_6_test_2696.wav   
2654  challenge_6_test_2697.wav   
2655  challenge_6_test_2698.wav   
2656  challenge_6_test_2699.wav   

     

In [17]:
# Ensembling
import pandas as pd
import numpy as np
from statistics import mean
import os

labels = {
    0: 'bird',
    1: 'eight',
    2: 'falcon',
    3: 'five',
    4: 'four',
    5: 'nine',
    6: 'one',
    7: 'seven',
    8: 'six',
    9: 'snake',
    10: 'three',
    11: 'two',
    12: 'zero'}

labels = {
    0: 'bird',
    1: 'cat',
    2: 'chicken',
    3: 'dog',
    4: 'down',
    5: 'eight',
    6: 'falcon',
    7: 'five',
    8: 'four',
    9: 'go',
    10: 'left',
    11: 'nine',
    12: 'one',
    13: 'right',
    14: 'seven',
    15: 'six',
    16: 'snake',
    17: 'stop',
    18: 'three',
    19: 'two',
    20: 'up',
    21: 'zero'}

labels = {
    0: 'backward',
    1: 'bird',
    2: 'cat',
    3: 'chicken',
    4: 'crocodile',
    5: 'dinosaur',
    6: 'dog',
    7: 'down',
    8: 'eight',
    9: 'elephant',
    10: 'falcon',
    11: 'five',
    12: 'forward',
    13: 'four',
    14: 'go',
    15: 'left',
    16: 'nine',
    17: 'one',
    18: 'right',
    19: 'seven',
    20: 'six',
    21: 'snake',
    22: 'stop',
    23: 'three',
    24: 'two',
    25: 'up',
    26: 'zero'}

df1 = pd.read_csv(r'softmax_outputs.csv', names=['file', 'softmax'])
df2 = pd.read_csv(r'softmax_outputs2.csv', names=['file', 'softmax'])
# df3 = pd.read_csv(r'softmax_outputs3.csv', names=['file', 'softmax'])
# df4 = pd.read_csv(r'softmax_outputs4.csv', names=['file', 'softmax'])
df_comb = df1.copy()
df_comb['softmax2'] = df2['softmax']
# df_comb['softmax3'] = df3['softmax']
# df_comb['softmax4'] = df4['softmax']

i = 0
def avg_softmax(input1, input2, input3='', input4=''):
    global i
    softmax1_list = list(input1)
    softmax2_list = list(input2)
    softmax3_list = list(input3)
    softmax4_list = list(input4)
    assert len(softmax1_list) == len(softmax2_list)
    softmax1 = softmax1_list[i].strip('[]').split()
    softmax2 = softmax2_list[i].strip('[]').split()[2:]
    # softmax3 = softmax3_list[i].strip('[]').split()
    # softmax4 = softmax4_list[i].strip('[]').split()
    softmax_avg = []
    for j in range(len(softmax1)):
        avg = mean([float(softmax1[j]), float(softmax2[j])])  # float(softmax3[j]), float(softmax4[j])
        softmax_avg += [avg]
    i += 1
    return softmax_avg


def get_label(softmax):
    global j
    if max(softmax) < 0.5:
        print(labels[np.argmax(softmax)])  # show low conf score
    return labels[np.argmax(softmax)]

df_avg = df_comb.apply(lambda avg: avg_softmax(df_comb['softmax'], df_comb['softmax2']), axis=1)  # df_comb['softmax3'], df_comb['softmax4']
avg_pred = df_avg.map(lambda label: get_label(label))
filenames = map(lambda x: x.replace('_normalized', ''), os.listdir('challenge_6_test_dataset_normalized/'))
df = pd.DataFrame(list(zip(filenames, avg_pred)))
df.to_csv('challenge_6_team_Tensor is not flowing.csv', index=False, header=False)
print(df)

cat
one
forward
crocodile
right
dinosaur
crocodile
dinosaur
four
bird
right
stop
snake
down
five
down
go
dog
bird
left
stop
seven
bird
unknown
six
six
backward
dinosaur
right
silence
cat
six
two
four
down
bird
stop
crocodile
go
seven
left
one
six
five
three
nine
crocodile
dog
eight
three
elephant
right
backward
four
left
crocodile
dog
eight
one
stop
silence
stop
two
snake
stop
crocodile
go
snake
dinosaur
silence
forward
six
right
three
four
nine
forward
five
eight
falcon
bird
elephant
backward
four
left
falcon
forward
dog
seven
right
down
dog
crocodile
cat
chicken
right
eight
unknown
forward
falcon
unknown
eight
silence
backward
dinosaur
eight
elephant
nine
elephant
dog
five
cat
one
snake
dinosaur
four
down
right
cat
forward
cat
three
dinosaur
three
three
falcon
crocodile
dinosaur
stop
five
down
silence
backward
snake
stop
left
stop
dinosaur
elephant
seven
silence
six
stop
snake
go
five
chicken
cat
nine
right
down
unknown
eight
eight
go
nine
forward
forward
one
forward
dog
left
elephan